# The basics
This jupyter notebook will make sure that you have everything you need to run langchain and explain a little bit the basics of interacting with GPT models.
Adapted from [LangChain's tutorial](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)

## Installing packages
We will use [LangChain](https://www.langchain.com/langchain) as our SDK to interact with different LLM's. It's abstractions concerning the different models as well as easy to plug-in vector DB's and adding "memory" to a use-case make it one of the best tools to prototype GenAI products. For that we need to make sure we have the right packages installed.

Quick note on jupyter-notebooks: 
* ctrl+enter will run the cell
* Any cell starting with `!` will me a command that you could also run in a terminal
* Feel free to modify the code inside them and play around with the results

In [ ]:
!pip install -r requirements.txt

# Calling our GPT model
We will use a model deployed on Azure. The way that we interact with the model is through a POST request to a specific endpoint. This is how the request looks like:
```json
{
  "temperature": 1,
  "top_p": 1,
  "stream": false,
  "stop": null,
  "max_tokens": 4096,
  "presence_penalty": 0,
  "frequency_penalty": 0,
  "logit_bias": {},
  "user": "user-1234",
  "messages": [
    {}
  ],
  "data_sources": [
    {}
  ],
  "n": 1,
  "seed": 1,
  "response_format": {
    "type": "json_object"
  },
  "tools": [
    {
      "type": "function",
      "function": {
        "description": "string",
        "name": "string",
        "parameters": {
          "additionalProp1": {}
        }
      }
    }
  ],
  "tool_choice": "none",
  "functions": [
    {
      "name": "string",
      "description": "string",
      "parameters": {
        "additionalProp1": {}
      }
    }
  ],
  "function_call": "none"
}
```
As you can see, there are different properties and we won't go through all of them on this workshop. The most important one will be the `messages` properties which contains all the different messages from the user as well as the answers from the model.

At this point, you could use whatever you want to interact with the LLM, for example, postman or curl from your terminal. This is one examle:

```shell
curl https://YOUR_RESOURCE_NAME.openai.azure.com/openai/deployments/YOUR_DEPLOYMENT_NAME/chat/completions?api-version=2024-02-01 \
  -H "Content-Type: application/json" \
  -H "api-key: YOUR_API_KEY" \
  -d '{"messages":[{"role": "user", "content": "hello!"}]}'
```

Please ask the moderator for the endpoint, API key and all the needed information to run the model.

## Using LangChain
The easiest way to start interacting with the API with python is to use LangChain's [AzureChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.azure.AzureChatOpenAI.html#langchain_openai.chat_models.azure.AzureChatOpenAI). This is object inherits from `ChatModels`. They are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. This allows us also to easily change of LLM without changing the code.

To just simply call the model, we can pass in a list of messages to the `.invoke()` method.

In [ ]:
from langchain_openai import AzureChatOpenAI

In [ ]:
azure_deployment=""
api_key=""
openai_api_version="2024-02-01"
azure_endpoint=""

In [ ]:
gpt_35 = AzureChatOpenAI(
    azure_deployment=azure_deployment,
    api_key=api_key,
    openai_api_version=openai_api_version,
    azure_endpoint=azure_endpoint
)

In [ ]:
gpt_35.invoke("hello!")

### Messages
We can also use messages to keep track of our inputs, and separate between `SystemMessage`, `HumanMessage` and `AIMessage`. For example:

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

gpt_35.invoke(messages)

### OutputParsers
Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser = StrOutputParser()

result = gpt_35.invoke(messages)
parser.invoke(result)

More commonly, we can "chain" the model with this output parser. This means this output parser will get called everytime in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [ ]:
chain =  gpt_35 | parser

In [ ]:
chain.invoke(messages)

### Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:

* `language`: The language to translate text into
* `text`: The text to translate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
prompt_template.messages

The input to this prompt template is a dictionary (a python JSON if you want...). We can play around with this prompt template by itself to see what it does by itself

In [ ]:
prompt_template.invoke({"language" : "french", "text": "hello!"})

We can now combine this with the model and the output parser from above. This will chain all three components together.

In [ ]:
chain = prompt_template | gpt_35 | parser

chain.invoke({"language": "french", "text": "hi"})

# Exercise!

Make a chain that will take as an input a superhero and an animal and returns a a creative name for the new superhero-animal.